## Backtesting
### Métricas (1)

En este cuaderno se presentan algunas métricas de rendimiento que podemos calcular sobre series temporales y en particular sobre el resultado de las estrategias de trading. 

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools

### Datos 
Asumiremos por un tema se simplicidad en los ejemplos, que nuestro universo de acciones invertibles son las siguientes acciones del IBEX35. 

In [ ]:
ticker_list = ['BBVA','SAN','REP','TEF','IBE','FER','ITX','ACS','AMS','GRF']

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_series = {ticker: df.close
                for ticker, df in stock_data.items()
                if ticker in ticker_list
               }
stock_df = pd.DataFrame(close_series)
stock_df = stock_df.loc['2010':]
stock_df.head()

____
### Evolución Relativa
La utilizamos para ver el rendimiento de varios activos o carteras a partir de un **punto temporal de referencia** que nos indicaría algún momento en la toma de decisión.  La clave es que todos los elementos comparados empiezan en 1, y la evolución de cada uno es 
el resultado de reproducir los rendimientos posteriores.  Esto puede conseguirse fácilmente sobre una serie de precios:
    1. Crear una serie a partir del punto de referencia
    2. Dividr esta serie entre su primer valor.

In [ ]:
def relative_serie(s):
    """ Calcula la serie relativa asumiendo el punto de referencia como el primer precio"""
    return s/s.iloc[0]

In [ ]:
relative_df = stock_df.apply(relative_serie)
relative_df.plot(figsize=(10,6))

Esta evolución la podríamos generar a partir de diferentes puntos de referencia.m

In [ ]:
for iyear in range(2010, 2021, 2):
    substock_df = stock_df.loc[f'{iyear}':]
    relative_df = substock_df.apply(relative_serie)
    relative_df.plot(figsize=(10,6))

___

### Drawdown y Tiempo bajo Agua

- El **Drawdown** es la pérdida máxima que se observa en una inversión entre dos puntos máximos de marcas de agua
- El **Tiempo bajo el Agua** (TuW) es el período de tiempo que pasa entre esos dos máximos 

In [ ]:
ticker = 'AMS'
stock_series = stock_df[ticker].dropna()

Respecto a una serie vemos sus máximos alcanzados (marcas de agua). La función *expanding* nos permite calcular la una función hasta cada último punto de la serie

In [ ]:
water_marks = stock_series.expanding().max()
# la funcion cummax() es equivalente

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
stock_series.plot(ax=ax1)
water_marks.plot(ax=ax2)

el drawdown lo calculamos como la rentabilidad hasta el último máximo.  De interés es:
 - la gráfica de los drawdowns
 - el máximo drawdown en un periodo de tiempo

In [ ]:
drawdowns = (stock_series/stock_series.cummax()) - 1
drawdowns.plot()

In [ ]:
## maximo drawdown
drawdowns.min()

In [ ]:
under_water = (stock_series < stock_series.cummax()).astype(float)
under_water

Las fechas de corte donde tendremos que calcular la suma de los días. Si la última no es máximo la consideramos también

In [ ]:
cut_uw = under_water[under_water == 0]
if cut_uw.index[-1] != under_water.index[-1]:
    cut_uw.loc[under_water.index[-1]] = 0
cut_uw

Ahora hacemos la suma acumulativa entre dos fechas de corte

In [ ]:
twu = pd.Series(np.zeros(under_water.shape), index=under_water.index)
current = cut_uw.index[0]
for idate in cut_uw.index[1:]:
    twu.loc[current:idate] = under_water[current:idate].cumsum()
    current = idate

In [ ]:
twu

In [ ]:
twu.plot()

Hacemos una función que nos calcule TuW todo a la vez

In [ ]:
def time_under_water(vseries):
    """Calcula una serie con el numero de sesiones 
    desde el ultimo maximo
    """
    under_water = (vseries < vseries.cummax()).astype(float)

    # fechas de corte
    cut_uw = under_water[under_water == 0]
    if cut_uw.index[-1] != under_water.index[-1]:
        cut_uw.loc[under_water.index[-1]] = 0
    
    # 
    twu = pd.Series(np.zeros(under_water.shape), index=under_water.index)
    current = cut_uw.index[0]
    for idate in cut_uw.index[1:]:
        twu.loc[current:idate] = under_water[current:idate].cumsum()
        current = idate
    return twu

____
Calculamos el time under water para los acciones que estamos trabajando en el ejemplo

In [ ]:
tuw_df = stock_df.apply(time_under_water)

In [ ]:
tuw_df.plot(figsize=(8,5))